# Import

In [111]:
import os
import re
import sys 
from datetime import datetime
from loguru import logger

import matplotlib.pyplot as plt
%matplotlib qt


import torch
import numpy as np
import cvxpy as cp
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False

import pyDRTtools as drt





# Loesner Framework

## Definition

In [112]:
def TikDRT(f, Z, RLC_Flag=[False, False, False], custom_lambda = None):
    '''==================================================
        Tikhonov DRT Deconvolution
        Parameter: 
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
            ch_eis: 3 x n_freq Matrix: [freq, Real, Imag]
        Returen:
            tau_vec: time domain vector
            x: DRT result
            n_extend: number of extend RLC parameters
        ==================================================
    '''
    ## Freq domain data prepare
    
    n_freq = len(f)
    # freq_vec = f
    # Z_exp = Z
    freq_vec = np.flip(f)
    Z_exp = np.flip(Z)

    '''Hyper Parameters'''
    # Time domain parameters
    log_tau_min = np.log10(1/(2*np.pi*freq_vec[0]))  
    log_tau_max = np.log10(1/(2*np.pi*freq_vec[-1])) 
    # log_tau_min = np.log10(1/(freq_vec[0]))  
    # log_tau_max = np.log10(1/(freq_vec[-1]))   
    n_tau = n_freq

    # tau_vec = 1/(2*np.pi*freq_vec)
    # tau_vec = 1/(2*np.pi*100*freq_vec)
    # tau_vec = np.logspace(-4,4, n_tau, endpoint=True)
    tau_vec = np.logspace(log_tau_min, log_tau_max, num = n_tau, endpoint=True)

    # log_tau = np.log(tau_vec)
    # tau_vec = np.logspace(-6, 0, n_tau, endpoint=True)
    # freq_vec = np.flip(np.logspace(0, 6, n_freq, endpoint=True))
    


    # Discretization matrices Parameters
    # Use RBF Kernel to initialize the A matrix
    RBF_shape_control = 'FWHM Coefficient' 
    RBF_coeff = 0.5
    # RBF_type = 'Piecewise Linear'
    RBF_type = 'Gaussian'
    # RBF_type = 'C0 Matern'
    # RBF_type = 'C2 Matern'
    # RBF_type = 'C4 Matern'
    # RBF_type = 'C6 Matern'
    # RBF_type = 'Inverse Quadratic'

    # Cross-validation Method for optimize lambda (Tikhonov regularization parameter) 
    cv_type = 'GCV'     # Generalized Cross Validation
    # cv_type = 'mGCV'    # Modified Generalized Cross Validation
    # cv_type = 'rGCV'    # Robust Generalized Cross Validation
    # cv_type = 'LC'      # L-curve
    # cv_type = 're-im'   # Real-Imaginary discrepancy
    # cv_type = 'kf'      # k-fold cross-validation

    '''Compute the RBF Shape Parameter Epsilon'''
    epsilon = drt.basics.compute_epsilon(freq_vec, RBF_coeff, RBF_type, RBF_shape_control)

    # logger.info(f"{epsilon}")
    # return
    '''Compute the discretization matrices'''
    A_re = drt.basics.assemble_A_re(freq_vec, tau_vec, epsilon, RBF_type)
    n_extend = np.sum(RLC_Flag)   
    if RLC_Flag[2]:
        A_re_C_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_C_0, A_re)) 
    if RLC_Flag[1]:
        A_re_L_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_L_0, A_re))
    if RLC_Flag[0]:
        A_re_R_inf  = np.ones((n_freq, 1))
        A_re        = np.hstack((A_re_R_inf, A_re)) 

    A_im = drt.basics.assemble_A_im(freq_vec, tau_vec, epsilon, RBF_type)
    if RLC_Flag[2]:
        A_im_C_0    = -1/(2*np.pi*freq_vec.reshape(-1,1))
        A_im        = np.hstack((A_im_C_0, A_im))
    if RLC_Flag[1]:
        A_im_L_0    = 2*np.pi*freq_vec.reshape(-1,1)
        A_im        = np.hstack((A_im_L_0, A_im))
    if RLC_Flag[0]:
        A_im_R_inf  = np.zeros((n_freq, 1)) 
        A_im        = np.hstack((A_im_R_inf, A_im))

    A = np.vstack((A_re, A_im))
     


    '''Compute the differentiation matrices for Tiknonov regularization'''
    M2 = np.zeros((n_tau+n_extend, n_tau+n_extend))
    M2[n_extend:,n_extend:] = drt.basics.assemble_M_2(tau_vec, epsilon, RBF_type)

    '''Optimize lambda'''
    if custom_lambda is None:
        log_lambda_init = -3 # ln(lambda_init = 0.001)
        # lambda_opt = drt.basics.optimal_lambda(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, "Combined Re-Im Data", RLC_Flag[1], log_lambda_init, cv_type)
        lambda_opt = drt.basics.optimal_lambda(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, "Combined Re-Im Data", 0, log_lambda_init, cv_type)
    else: 
        lambda_opt = custom_lambda
    # logger.info(f"Lambda: {lambda_opt}")
    '''Deconvolve The DRT from the EIS Data'''
    # Set Bound Constraints
    # lb = np.zeros([n_tau+n_extend])
    # bound_mat = np.eye(lb.shape[0])
    H_combined, c_combined = drt.basics.quad_format_combined(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, lambda_opt)
    G = matrix(-np.identity(Z_exp.shape[0]+n_extend))
    h = matrix(np.zeros(Z_exp.shape[0]+n_extend))


    # Deconvolved DRT - Old
    sol = solvers.qp(matrix(H_combined), matrix(c_combined), G, h)
    x = np.array(sol['x'])

    # Deconvolved DRT - New
    # x_var = cp.Variable(H_combined.shape[0])
    # objective = cp.Minimize(0.5 * cp.quad_form(x_var, H_combined) + c_combined.T @ x_var)
    # constraints = [x_var >= 0]
    # prob = cp.Problem(objective, constraints)
    # prob.solve(solver=cp.ECOS, verbose=False,
    #        abstol=1e-6, reltol=1e-6)

    # x = x_var.value

    # Output layer
    H = A@x
    H = H[:n_freq] + 1j*H[n_freq:]
    H = np.flip(H).flatten()
    R_i = x[n_extend:].flatten() / 2
    C_i = tau_vec/R_i

    return R_i, C_i, tau_vec, H, x[:n_extend].flatten(), lambda_opt



def DRT_Analysis_Batch(chData, RLC_Flag=[False, False, False], custom_lambda = None):
    '''==================================================
        DRT Analysis for Batch Data
        Parameter: 
            chData:     list of tuples (f, Z) for each channel
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            results:    list of tuples (R_i, C_i, tau_i, H, res_ReZ, res_ImZ) for each channel
        ==================================================
    '''
    DRTdata = []
    RC_list = []
    H_list = []
    f = chData[0,0,:]
    for i in range(chData.shape[0]):
        _Z = chData[i,1,:] + 1j*chData[i,2,:]
        R_i, C_i, tau_i, H, RC, _= TikDRT(f, _Z, RLC_Flag, custom_lambda)
        DRTdata.append((np.array([R_i, C_i, tau_i])))
        RC_list.append(RC)
        H_list.append(H)
    return DRTdata, RC_list, H_list

def DRT_Res(Z, H):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return res_ReZ, res_ImZ




def DRT_Single_Plot(f, Z_plot, H_plot, R_plot, C_plot, tau_plot, res_ReZ_plot=None, res_ImZ_plot=None):
    '''==================================================
        Plot DRT Analysis Results
        Parameter: 
            f:              real array of frequency values
            Z_plot:         complex array of impedance values (H = Z)
            H_plot:         reconstructed impedance values
            R_plot:         R_i from RC pair in DRT
            tau_plot:       tau_i from RC pair in DRT
            res_ReZ_plot:   relative error of real part of impedance values
            res_ImZ_plot:   relative error of imaginary part of impedance values
            svd_L_plot:     singular values from Loewner Pencel
        ==================================================
    '''

    fig, axis = plt.subplots(2,3,figsize=(12,8), constrained_layout=True)
    cmp = plt.colormaps.get_cmap('rainbow_r')
    axis[0,0].plot(Z_plot.real, -Z_plot.imag, label = "EIS")
    axis[0,0].plot(H_plot.real, -H_plot.imag, label = "DRT")
    axis[0,0].set_aspect('equal')


    axis[0,1].loglog(f, np.abs(Z_plot), label = "EIS")
    axis[0,1].loglog(f, np.abs(H_plot), label = "DRT")
    axis[0,2].semilogx(f, -np.angle(Z_plot), label = "EIS")
    axis[0,2].semilogx(f, -np.angle(H_plot), label = "DRT")

    axis[1,0].stem(tau_plot, R_plot, linefmt='-.', markerfmt='o', basefmt='-')
    # axis[1,0].plot(tau_plot, R_plot)
    axis[1,0].set_xscale('log')
    # axis[1,0].set_yscale('log')
    axis[1,0].set_xlabel(r'$\tau_i\ [s]$')
    axis[1,0].set_ylabel(r'$R_i\ [\Omega]$')  

    # 右轴：C_plot
    ax2 = axis[1,0].twinx()  # 创建共享x轴的右侧y轴
    ax2.stem(tau_plot, C_plot, basefmt='-', linefmt='C1-.', markerfmt='C1x')
    # ax2.plot(tau_plot, 1/C_plot, 'orange')
    ax2.set_ylabel(r'$C_i\ [F]$')
    # ax2.set_yscale('log')

    if res_ReZ_plot is not None:
        axis[1,1].plot(f,res_ReZ_plot, ':+', label = "Re(Z)")
        
    if res_ImZ_plot is not None:
        axis[1,1].plot(f,res_ImZ_plot, '--o', label = "Im(Z)")
    axis[1,1].set_xscale('log')


    return fig



def DRT_Plot_Batch(fig, DRTdata, chData):
    # axis = [0] * 6
    # axis[0] = fig.add_axes([0.0625,0.5,0.25,0.4])
    # axis[1] = fig.add_axes([0.375,0.5,0.25,0.4])
    # axis[2] = fig.add_axes([0.6875,0.5,0.25,0.4])
    

    # axis[3] = fig.add_axes([0.0625,0.05,0.25,0.4])
    # axis[4] = fig.add_axes([0.375,0.05,0.25,0.4])
    # axis[5] = fig.add_axes([0.6875,0.05,0.25,0.4])


    axis = [0] * 5
    axis[0] = fig.add_axes([0.0625,0.5,0.25,0.4])
    axis[1] = fig.add_axes([0.375,0.5,0.25,0.4])
    axis[2] = fig.add_axes([0.6875,0.5,0.25,0.4])
    
    axis[3] = fig.add_axes([0.0625,0.05,0.4,0.4])
    axis[4] = fig.add_axes([0.5375,0.05,0.4,0.4])



    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(chData.shape[0]):
        ch_eis = chData[i,:,:]
        ch_drt = DRTdata[i]
        _color = cmap(i/chData.shape[0])

        axis[0].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2)
        axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2)
        axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2)
        # axis[3].scatter(i, LambdaList[i], color = _color)
        
        axis[3].plot(ch_drt[2,:], ch_drt[0,:], color=_color, linewidth=2, alpha=0.5)
        axis[4].plot(ch_drt[2,:], 1/ch_drt[1,:], color=_color, linewidth=2, alpha=0.5)


    axis[0].set_aspect('equal', adjustable='datalim')
    # axis[3].set_yscale('log')

    axis[3].set_xscale('log')
    axis[3].set_yscale('log')
    axis[4].set_xscale('log')
    axis[4].set_yscale('log')





def DRT_Plot_Batch_3D(fig, DRTdata, chData):
    axis = [0] * 5
    axis[0] = fig.add_axes([0.0625,0.5,0.25,0.4])
    axis[1] = fig.add_axes([0.375,0.5,0.25,0.4])
    axis[2] = fig.add_axes([0.6875,0.5,0.25,0.4])
    
    axis[3] = fig.add_axes([0.0625,0.05,0.4,0.4], projection='3d')
    axis[4] = fig.add_axes([0.5375,0.05,0.4,0.4], projection='3d')

    z_offset = 0.2
    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(chData.shape[0]):
        ch_eis = chData[i,:,:]
        ch_drt = DRTdata[i]
        _color = cmap(i/chData.shape[0])

        axis[0].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2)
        axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2)
        axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2)

        z = np.ones_like(ch_drt[2, :]) * i * z_offset
        x = np.log10(ch_drt[2, :])  
        y1 = np.log10(ch_drt[0, :])  # R_i
        y2 = np.log10(ch_drt[1, :])  # C_i

        # for xi, yi, zi in zip(x, y1, z):
        #     axis[3].plot([xi, xi], [zi, zi], [0, yi], color=_color, linewidth=1.5)
        # for xi, yi, zi in zip(x, y2, z):
        #     axis[4].plot([xi, xi], [zi, zi], [0, yi], color=_color, linewidth=1.5)

        axis[3].scatter(x, z, y1, color=_color, s=20)
        axis[4].scatter(x, z, y2, color=_color, s=20)

        

    axis[0].set_aspect('equal', adjustable='datalim')




## Simulation Data

### R-(R||C)-(R||C)

In [113]:
# # R-(R||C)-(R||C)
# # Element
# R1 = 200; # Ω
# R2 = 100; # Ω
# R0 = 70;  # Ω

# C1 = 2.5e-3; # 
# C2 = 1e-4;   # 

# # tau1 = R1*C1;  # s
# # tau2 = R2*C2;  # s

# f = np.logspace(-2,2,41);  # Hz

# # Calculation of the impedance dataset 
# Z_sim = np.array([1/((1/R1)+1j*w*C1) +1/((1/R2)+1j*w*C2) +R0 for w in 2*np.pi*f])
# Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim




### Randle

In [114]:
# R0 = 70

# R1 = 10000 
# C1 = 2.5e-9

# Y1 = 1e-5 
 
# n1 = 0.66
# f = np.logspace(-1,5,61);  # Hz 

# Q1 = lambda x: 1/(Y1*(1j*x)**n1)
# Z_sim = np.array([ R0 + (R1+Q1(w))/(1+1j*w*C1*(R1+Q1(w))) for w in 2*np.pi*f])
# Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim

### Run LF

In [115]:
# R_i, C_i, tau_i, H, A, lambda_i = TikDRT(f, Z_sim_noise, RLC_Flag=[True, False, False])
# res_ReZ, res_ImZ = DRT_Res(Z_sim_noise, H)


# # R_n, C_n, tau_n, H, A, lambda_n = TikDRT(f, Z_sim_noise)


### Plot

In [116]:
# Z_plot = Z_sim_noise
# H_plot = H
# R_plot = R_i
# C_plot = C_i
# tau_plot = tau_i

# res_ReZ_plot = res_ReZ
# res_ImZ_plot = res_ImZ
# DRT_Single_Plot(f, Z_plot, H_plot, R_plot, C_plot, tau_plot,  res_ReZ_plot, res_ImZ_plot)

In [117]:
# Z_plot = Z_sim_noise
# H_plot = H_n
# R_plot = R_n
# C_plot = C_n
# tau_plot = tau_n
# res_ReZ_plot = res_ReZ_n
# res_ImZ_plot = res_ImZ_n
# DRT_Single_Plot(f, Z_plot, H_plot, R_plot, C_plot, tau_plot, res_ReZ_plot, res_ImZ_plot)

In [118]:
# tau_r = np.abs(tau_n.real / tau_n.imag)
# plt.figure()
# ax1 = plt.subplot(211)
# ax1.loglog(tau_i, R_i, 'o', label='R_i')
# ax1.loglog(tau_n[tau_r>1].real, R_n[tau_r>1].real, '*', label='R_n')
# ax1.legend()

# ax2 = plt.subplot(212)  
# ax2.loglog(tau_i, C_i, 'ro', label='C_i')
# ax2.loglog(tau_n[tau_r>1], C_n[tau_r>1], '*', label='C_n')
# ax2.legend()


# Data Loader

## Definition

In [119]:
def SearchELE(rootPath, ele_pattern = re.compile(r"(.+?)_归档")):
    '''==================================================
        Search all electrode directories in the rootPath
        Parameter: 
            rootPath: current search path
            ele_pattern: electrode dir name patten
        Returen:
            ele_list: list of electrode directories
        ==================================================
    '''
    ele_list = []
    for i in os.listdir(rootPath):
        _path = os.path.join(rootPath, i)
        if os.path.isdir(_path):
            match_ele = ele_pattern.match(i)
            if match_ele:
                ele_list.append([_path, match_ele.group(1)])
            else:
                ele_list.extend(SearchELE(_path, ele_pattern))

    return ele_list

In [120]:
def setup_logger(log_dir="./LOG", log_filename="file.log", file_level="WARNING", console_level="WARNING"):
    # 创建目录
    os.makedirs(log_dir, exist_ok=True)
    log_fd = os.path.join(log_dir, log_filename)

    logger.remove()
    # 如果已有日志文件，重命名添加时间戳
    if os.path.exists(log_fd):
        name, ext = os.path.splitext(log_filename)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archived_name = f"{name}_{timestamp}{ext}"
        archived_path = os.path.join(log_dir, archived_name)
        os.rename(log_fd, archived_path)

    # 添加终端输出
    logger.add(sys.stdout, level=console_level, enqueue=True)

    # 添加文件输出
    logger.add(log_fd, level=file_level, encoding="utf-8", enqueue=True)

    return logger

## Run

In [121]:

# rootPath = "D:/Baihm/EISNN/Archive/"
# ele_list = SearchELE(rootPath)
# DATASET_SUFFIX = "Outlier_Ver04"
# freq_list = np.linspace(1000,5000-1,101,dtype=int, endpoint=True)
# freq_list = freq_list[1:]


# rootPath = "D:/Baihm/EISNN/Archive_New/"
# ele_list = SearchELE(rootPath)
# DATASET_SUFFIX = "Outlier_Ver04"
# freq_list = np.linspace(1000,5000-1,101,dtype=int, endpoint=True)
# freq_list = freq_list[1:]


rootPath = "D:/Baihm/EISNN/Invivo/"
ele_list = SearchELE(rootPath, re.compile(r"(.+?)_Ver02"))
DATASET_SUFFIX = "Outlier_Ver04"
freq_list = np.linspace(0,5000-1,101,dtype=int, endpoint=True)
freq_list = freq_list[1:]


n_ele = len(ele_list)
logger.info(f"Search in {rootPath} and find {n_ele:03d} electrodes")

In [122]:
if True:
    setup_logger(log_dir="D:\Baihm\EISNN\LOG\DRT_Process_Log")

# logger.remove()
# logger.add(sys.stdout, level="WARNING")
# logger.add("./LOG/file.log", rotation="10 MB", level="INFO")

2025-07-05 23:50:13.074 | WARNING  | __main__:<module>:21 - ELE[1/5]: 	S5877 - D:/Baihm/EISNN/Invivo\S5877_Ver02
2025-07-06 00:01:56.943 | WARNING  | __main__:<module>:21 - ELE[2/5]: 	S6005 - D:/Baihm/EISNN/Invivo\S6005_Ver02
2025-07-06 00:12:10.753 | WARNING  | __main__:<module>:21 - ELE[3/5]: 	S6006 - D:/Baihm/EISNN/Invivo\S6006_Ver02
2025-07-06 00:22:38.453 | WARNING  | __main__:<module>:21 - ELE[4/5]: 	S6072 - D:/Baihm/EISNN/Invivo\S6072_Ver02
2025-07-06 00:31:04.882 | WARNING  | __main__:<module>:21 - ELE[5/5]: 	S6106 - D:/Baihm/EISNN/Invivo\S6106_Ver02


In [123]:


DRT_SUFFIX = f"{DATASET_SUFFIX}_DRTTik_Ver01"
SAVE_FLAG = True
FORCE_FLAG = False

In [124]:

for i in range(n_ele):
# for i in range(0,1):
    # logger.info(f"ELE Begin: {ele_list[i][0]}")
    fd_pt = os.path.join(ele_list[i][0], DATASET_SUFFIX, f"{ele_list[i][1]}_{DATASET_SUFFIX}.pt")
    if not os.path.exists(fd_pt):
        logger.warning(f"{fd_pt} does not exist")
        continue
    
    logger.debug(fd_pt)
    data_pt = torch.load(fd_pt)
    _meta_group = data_pt["meta_group"]
    _data_group = data_pt["data_group"]


    ele_id  = _meta_group["ele_id"]
    elePath = _meta_group["elePath"]
    n_ch = _meta_group["n_ch"]      
    x_day_full = _meta_group["TimeSpan"]


    logger.warning(f"ELE[{i+1}/{n_ele}]: \t{ele_id} - {elePath}")


    # Storage path
    save_dir = f"{elePath}/{DRT_SUFFIX}/"
    pt_file_name = f"{ele_id}_{DRT_SUFFIX}.pt"
    os.makedirs(save_dir, exist_ok=True)
    if os.path.exists(os.path.join(save_dir, pt_file_name)):
        logger.warning(f"FileAlreadyExistsWarning: {ele_id} - {pt_file_name} already exists.")
        if SAVE_FLAG and not FORCE_FLAG:
            continue


    for j in _data_group['Channels']:
        try:
        # if 1:
            logger.info(f"ELE[{ele_id}] - ch[{j:03d}] Begin") 
            channel_group_raw = _data_group[j]

            
            chData      = channel_group_raw['chData']         
            eis_seq     = channel_group_raw['eis_seq']        
            eis_cluster = channel_group_raw['eis_cluster']    
            eis_anomaly = channel_group_raw['eis_anomaly']    

            if chData.shape[2] != 5000:
                logger.error(f"ELE[{ele_id}] - ch[{j}] with less than 5000 samples")
                break
            
            chData = chData[:, :, freq_list]
            chData = chData[eis_seq, :, :]

            # Run DRT Analysis
            
            DRTdata, RCList, HList = DRT_Analysis_Batch(chData, RLC_Flag=[True, False, True], custom_lambda = 1e-3)
            # logger.info(f"ELE[{ele_id}] - ch[{j:03d}] DRT Finished")
            
            # Plot DRT Analysis Results
            fig = plt.figure(figsize=(16, 9), constrained_layout=True)
            DRT_Plot_Batch(fig, DRTdata, chData)
            fig.suptitle(f"ELE[{ele_id}] - ch[{j:03d}] DRT Analysis", fontsize=16, fontweight='bold')


            # Save Fig
            fig_name = f"DRT_{ele_id}_ch{j:03d}.png"
            
            os.makedirs(save_dir, exist_ok=True) 
            path = os.path.join(save_dir, fig_name)

            fig.savefig(path)
            plt.close(fig) 

            # Data Saving
            channel_group_DRT = {}
              
              
            channel_group_DRT['chData']   = chData
            channel_group_DRT['DRTdata']  = DRTdata
            channel_group_DRT['RCData']  = RCList
            channel_group_DRT['HData']  = HList
            

            _data_group[j] = channel_group_DRT
            logger.info(f"ELE[{ele_id}] - ch[{j:03d}] Finished")
            

        except Exception as e:
            logger.error(f"ELE[{ele_id}] - ch[{j:03d}] Run with error: {e}")
            continue

    
    pt_store = {}
    pt_store["meta_group"] = _meta_group
    pt_store["data_group"] = _data_group
    if SAVE_FLAG:
        torch.save(pt_store, os.path.join(save_dir, pt_file_name))




C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_14160\1631189875.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_pt = torch.load(fd_pt)
C:\Users\Baihm\AppData\Local\

# Viewer

In [125]:
VIEWER_FLAG = False
if VIEWER_FLAG:
    # ele_id = "02067447"
    # ch_id = 16    # Short with two phased
    # ch_id = 68    # Short Type II

    
    ele_id = "06017758"
    ch_id = 10    # Perfect

    
    # ele_id = "01037160"
    # ch_id = 20    # Short type I

    
    # ele_id = "11057712"
    # # ch_id = 26      # Normal to Short
    # ch_id = 107      # Normal to Short
    
    fd_pt = f"D:/Baihm/EISNN/Archive/{ele_id}_归档\\Outlier_Ver04_DRT_Ver01\\{ele_id}_Outlier_Ver04_DRT_Ver01.pt"

    # Read out
    data_pt = torch.load(fd_pt)
    _data_group = data_pt["data_group"]
    chData = _data_group[ch_id]['chData']
    DRTdata = _data_group[ch_id]['DRTdata']
    
    # Plot
    fig = plt.figure(figsize=(16, 9), constrained_layout=True)
    DRT_Plot_Batch(fig, DRTdata[:], chData[:,:,:])
    # DRT_Plot_Batch_3D(fig, DRTdata[:2], chData[:2,:,:])



In [126]:
# fig = plt.figure(figsize=(16, 9), constrained_layout=True)
            
# DRT_Plot_Batch_3D(fig, DRTdata, chData)
            